In [105]:
import requests
import bs4
import psycopg2
import configparser

In [114]:
def get_connection():

        config = configparser.ConfigParser()
        config.read_file(open(r'config.ini'))

        DSN = config.get("DB", "host")
        DB = config.get("DB", "database")
        UID = config.get("DB", "user")
        PWD = config.get("DB", "password")

        conn = psycopg2.connect(
                    host=DSN,
                    database=DB,
                    user=UID,
                    password=PWD)
        return conn

def insert_data(book_title):
    conn = get_connection()
    cur = conn.cursor()

    insertsql = f'''INSERT INTO book_info(title, price) VALUES ('{book_title[0]}', '{book_title[1]}');'''

    cur.execute(insertsql)
    conn.commit()
    cur.close()
    conn.close()

def truncate_table():
    conn = get_connection()
    cur = conn.cursor()

    truncatesql = f'''TRUNCATE TABLE book_info;'''
    cur.execute(truncatesql)

    conn.commit()
    cur.close()
    conn.close()


In [135]:
base_url = 'http://books.toscrape.com/catalogue/page-{}.html'
# two_star_titles = []
truncate_table()
for i in range(1,51):
    scrape_url = base_url.format(i)
    response = requests.get(scrape_url)

    soup = bs4.BeautifulSoup(response.text, 'lxml')
    books = soup.select('.product_pod')

    for book in books:
        two_star_titles = []
        if len(book.select('.star-rating.Three')) != 0:
            two_star_titles.append(book.select('a')[1]['title'].replace("'",""))
            two_star_titles.append(book.select('p')[1].text.replace("Â",""))
            insert_data(two_star_titles)



In [131]:
# base_url = 'http://books.toscrape.com/catalogue/page-{}.html'
#
# scrape_url = base_url.format(1)
# response = requests.get(scrape_url)
#
# soup = bs4.BeautifulSoup(response.text, 'lxml')
# books = soup.select('.product_pod')
#
# books[0].select('p')[1].text

'Â£51.77'